##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/io/tutorials/genome"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.orgで表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/io/tutorials/genome.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/io/tutorials/genome.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub でソースを表示</a></td>
      <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/io/tutorials/genome.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

## 概要

このチュートリアルでは、一般的に使用されるゲノミクス IO 機能を提供する<code>tfio.genome</code>パッケージについて解説します。これは、いくつかのゲノミクスファイル形式を読み取り、データを準備するための一般的な演算を提供します (例: One-Hot エンコーディングまたは Phred クオリティスコアを確率に解析します)。

このパッケージは、[Google Nucleus](https://github.com/google/nucleus) ライブラリを使用して、主な機能の一部を提供します。 

## セットアップ

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
!pip install tensorflow-io

In [ ]:
import tensorflow_io as tfio
import tensorflow as tf

## FASTQ データ

FASTQ は、基本的な品質情報に加えて両方の配列情報を保存する一般的なゲノミクスファイル形式です。

まず、サンプルの`fastq`ファイルをダウンロードします。

In [ ]:
# Download some sample data:
!curl -OL https://raw.githubusercontent.com/tensorflow/io/master/tests/test_genome/test.fastq

### FASTQ データの読み込み

`tfio.genome.read_fastq`を使用してこのファイルを読みこみます (`tf.data` API は近日中にリリースされる予定です)。

In [ ]:
fastq_data = tfio.genome.read_fastq(filename="test.fastq")
print(fastq_data.sequences)
print(fastq_data.raw_quality)

ご覧のとおり、返された`fastq_data`には fastq ファイル内のすべてのシーケンスの文字列テンソル (それぞれ異なるサイズにすることが可能) である`fastq_data.sequences`、および、シーケンスで読み取られた各塩基の品質に関する Phred エンコードされた品質情報を含む`fastq_data.raw_quality`が含まれています。

### 品質

関心がある場合は、ヘルパーオペレーションを使用して、この品質情報を確率に変換できます。

In [ ]:
quality = tfio.genome.phred_sequences_to_probability(fastq_data.raw_quality)
print(quality.shape)
print(quality.row_lengths().numpy())
print(quality)

### One-Hot エンコーディング

また、One-Hot エンコーダ―を使用してゲノムシーケンスデータ (`A` `T` `C` `G`の塩基配列で構成される) をエンコードすることもできます。これに役立つ演算が組み込まれています。


In [ ]:
one_hot = tfio.genome.sequences_to_onehot(fastq_data.sequences)
print(one_hot)
print(one_hot.shape)

In [ ]:
print(tfio.genome.sequences_to_onehot.__doc__)